In [ ]:
# import library
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pulp
import random
import math
import pdb

In [ ]:
def read_network():
    '''
        input: network.dat
        output:
            num_zones: number of zones within the network
            num_nodes: number of nodes in the network
            num_links: number of links in the network
            node_detail: node id and zone which the node belong to
            node_id: match the node id to 1~num_nodes to save memory
            link_detail:The information associated with a specific link
                link_detail[0] upstream node
                link_detail[1] downstream node
                link_detail[2] Num of left turn bay
                link_detail[3] Num of right turn bay
                link_detail[4] length of link
                link_detail[5] Num of lanes
                link_detail[6] Traffic flow mode
                link_detail[7] posted speed limit adjustment margin
                link_detail[8] posted speed limit
                link_detail[9] maximum service flow rate for the first link
            link_id: a matrix to find a specific link based on origin node nd destination node 
   '''
    global num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id
    file=open('network.dat','r')
    i=0
    network_basic=list()
    node_list=list()
    node_id={}
    link_detail_list=list()
    for line in file:
        line_list_temp=line.split()
        # Read the first line of network.dat
        if i==0:
            num_zones=int(line_list_temp[0])
            num_nodes=int(line_list_temp[1])
            num_links=int(line_list_temp[2])
            num_shortest_path=int(line_list_temp[3])
            zone_flag=int(line_list_temp[4])
            link_id=np.zeros((num_nodes,num_nodes))
        #Read the node information from network.dat
        elif len(line_list_temp)<3:
            node_list.append([int(line_list_temp[0]),int(line_list_temp[1])])
            node_id.update({int(line_list_temp[0]):i-1})
        #Read the link information from network.dat
        else:
#             for j in line_list_temp:
#                 if '+' in j and len(j)>2:
#                     check=j.split('+')
#                     line_list_temp.pop(6) 
#                     line_list_temp.insert(6,check[0])
#                     line_list_temp.insert(7,check[1])
            line_list_temp=[float(j) for j in line_list_temp]
            link_detail_list.append(line_list_temp)
            link_id[node_id[line_list_temp[0]],node_id[line_list_temp[1]]]=i-num_nodes-1
        i=i+1
        
    node_detail=np.matrix(node_list)
    link_detail=np.matrix(link_detail_list)
    
    return  num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id
def read_flow():
    '''
        input: outflow.dat
        output: 
            link_volume - matrix
    '''
    global link_volume
    volume_time=[]
    file=open('OutFlow.dat')
    i=1
    link_volume=[]
    link_volume_one_time_interval=[]
    for line in file:
        line_list=line.split()
        if i>6 : # skip the first 6 lines
            if len(line_list)==1: #find the line indicating time interval
                volume_time.append(float(line_list[0]))
                link_volume.append(link_volume_one_time_interval)
                link_volume_one_time_interval=[]
            else:
                line_list=[float(j) for j in line_list]
                link_volume_one_time_interval.extend(line_list)
        i=i+1
    link_volume.append(link_volume_one_time_interval)
    link_volume.pop(0)
    link_volume=np.array(link_volume)
    return link_volume
def read_speed():
    '''
        input:OutLinkSpeedAll.dat
        output: 
            link_speed -matrix
    '''
    global link_speed
    speed_time=[]
    file=open('OutLinkSpeedAll.dat')
    i=1
    link_speed=[]
    link_speed_one_time_interval=[]
    for line in file:
        line_list=line.split()
        if i>6 : # skip the first 6 lines
            if len(line_list)==1: #find the line indicating time interval
                speed_time.append(float(line_list[0]))
                link_speed.append(link_speed_one_time_interval)
                link_speed_one_time_interval=[]
            else:
                line_list=[float(j) for j in line_list]
                link_speed_one_time_interval.extend(line_list)
        i=i+1
    link_speed.append(link_speed_one_time_interval)
    link_speed.pop(0)
    link_speed=np.array(link_speed)
    return link_speed


def read_xy():
    '''
        input: xy.dat
        output: nodexy: a dictionary storing the latitude and longtitude of 
            each node
    '''
    global nodexy
    nodexy={}
    file=open('xy.dat')
    for line in file:
        line_list_temp=line.split()
        line_list_temp=[float(j) for j in line_list_temp]
        nodexy.update({line_list_temp[0]:[line_list_temp[1]/1000000.0*51.33,-line_list_temp[2]/1000000.0*68]})
    return

def read_snow():
    '''
        ipnut: weather.dat
        output: snow_detail. 
                    snow_detail[i,0] snow intensity at time interval i
                    snow_detail[i,1] start time of time interval i
                    snow_detail[i,2] end time of time interval i
    '''
    global snow_detail,snow_interval
    file=open('weather.dat')
    snow_detail=[]
    i=0
    for line in file: 
        if i==0:
            num_interval=float(line.split()[0])
            #print(num_interval,i,line)
            snow_interval=planning_horizon/num_interval
        elif i!= num_interval+1:
            line_list_temp= line.split()
            snow_detail.append([float(line_list_temp[2]),float(line_list_temp[3]),float(line_list_temp[4])])
        i=i+1
    return
def read_snowaccum(): 
    global sa_factors
    sa_factors=[]
    file=open('SnowAccuFactor.dat')
    for line in file: 
        if len(line)>4:
            line_list_temp=[j for j in line.split()]
            sa_factors.append([float(j) for j in line_list_temp])
    return
# def read_scenario()
#     '''
#         Input: Scenario.dat
#         output:
#     '''

In [ ]:
def link_class_partrition():
    '''
        input: 
            link_volume
        output:
            link_class
    '''
    global link_class
    #Calculate the threshold of different service level
    service_threshold=[]
    service_threshold.append(np.percentile(np.amax(link_volume,axis=0),80))
    service_threshold.append(np.percentile(np.amax(link_volume,axis=0),40))
    service_threshold.append(np.percentile(np.amax(link_volume,axis=0),0))
    i=0
    link_class=[]
    for link in link_detail:
        if max(link_volume[:,i])>=service_threshold[0] or (link[11]!=5) :
            link_class.append(1)
        elif max(link_volume[:,i])>=service_threshold[1]: #or if it is a bus routes, need to modify later
            link_class.append(2)
        else: 
            link_class.append(3)
        i=i+1
    return
            

In [ ]:
def distance_between_link(link_ID1,link_ID2,link_xy):
#     xyoflink1=[]
#     xyoflink1.extend([(nodexy[link_detail[link_ID1,0]][0]+nodexy[link_detail[link_ID1,1]][0])/2])
#     xyoflink1.extend([(nodexy[link_detail[link_ID1,0]][1]+nodexy[link_detail[link_ID1,1]][1])/2])
#     xyoflink2=[]
#     xyoflink2.extend([(nodexy[link_detail[link_ID2,0]][0]+nodexy[link_detail[link_ID2,1]][0])/2])
#     xyoflink2.extend([(nodexy[link_detail[link_ID2,0]][1]+nodexy[link_detail[link_ID2,1]][1])/2])
#     distance=math.sqrt((xyoflink1[0]-xyoflink2[0])**2+(xyoflink1[0]-xyoflink2[0])**2)
    distance=math.sqrt((link_xy[link_ID1][0]-link_xy[link_ID2][0])**2+(link_xy[link_ID1][1]-link_xy[link_ID2][1])**2)
    return distance
def c_link_xy (links):
    global link_xy
    link_xy=[]
    for link_ID1 in links: 
        link_xy.append([(nodexy[link_detail[link_ID1,0]][0]+nodexy[link_detail[link_ID1,1]][0])/2,
                       (nodexy[link_detail[link_ID1,0]][1]+nodexy[link_detail[link_ID1,1]][1])/2])
    return 

In [ ]:
def net_work_partrition(network_link_list,num_cluster,center,seed_links):
    '''
    input: 
        network_link_list: a list of link id that needs to be partitioned 
        num_cluster: the network would be partitioned into num_cluster sets
        center: optional input, the location of the depot
        seed_links: optional input, the seed link for each cluster
    output
        clusters: a list of link contained in each cluster
    '''
    #global d
    if not center: 
        centertemp=random.choice(network_link_list)
        center=link_detail[centertemp,0]
    if not seed_links: # in this case, we do not have a seedlink,but have a center depot
        for i in range(num_cluster):
            maxdistance=0
            linkcandidate=0
            #Go over all the candidate links to find the one which locates furtherest to the depot and other seedlinks
            for link in network_link_list:
                #calculate distance form link candidate to the depot
                distance_to_depot=math.sqrt((nodexy[link_detail[link,0]][0]-nodexy[center][0])**2
                                            +(nodexy[link_detail[link,0]][1]-nodexy[center][1])**2)
                total_distance=distance_to_depot
                #calculate distance form link candidate to other seed links
                for seed_link in seed_links: 
                    total_distance=total_distance*distance_between_link(seed_link,link,link_xy)
                if total_distance>maxdistance and link not in seed_links:
                    maxdistance=total_distance
                    linkcandidate=link
            seed_links.append(linkcandidate)
    print('The following links are selected as seed link: \n',seed_links)
    
    #After the seed_links are selected, solve the integer problem to identify the link clusters
    cluster_link_index=[i for i in range(len(network_link_list))]
    cluster_zone_index=[i for i in range(num_cluster)]
    link_length=[link_detail[i,4] for i in network_link_list]
    
    L=float(sum(link_length)/num_cluster*0.95)
    U=float(sum(link_length)/num_cluster*1.05)

    prob=pulp.LpProblem('Link Cluster Problem',pulp.LpMinimize)
    link_cluster=pulp.LpVariable.dicts("Cluster",(cluster_link_index,cluster_zone_index),0,1,pulp.LpInteger)
    d=np.zeros(shape=(len(network_link_list),num_cluster))
    for l in cluster_link_index:
        for z in cluster_zone_index:
            #print(l,z,network_link_list[l],seed_links[z])
            d[l,z]=distance_between_link(network_link_list[l],seed_links[z],link_xy)
            
    prob += pulp.lpSum([d[l,z]*link_cluster[l][z]] for l in cluster_link_index for z in cluster_zone_index), "Total distance from link to assigned seed link"
    for l in cluster_link_index:
        prob += pulp.lpSum([link_cluster[l][z] for z in cluster_zone_index]) == 1
    for z in cluster_zone_index:
        prob += pulp.lpSum([float(link_length[l])*link_cluster[l][z]] for l in cluster_link_index )>=L
        prob += pulp.lpSum([float(link_length[l])*link_cluster[l][z]] for l in cluster_link_index)<=U
        
    prob.writeLP("Link Cluster Problem.lp")
    # The problem is solved using PuLP's choice of Solver
    prob.solve()
    print("Solution Status:", pulp.LpStatus[prob.status])
    network_link_cluster_list=[[] for i in range(num_cluster)]
    for l in cluster_link_index:
        for z in cluster_zone_index:
            if pulp.value(link_cluster[l][z])==1:
                network_link_cluster_list[z].append(network_link_list[l])
    
    return network_link_cluster_list




In [ ]:
def network_partrition_nearest_heuristic(network_link_list,num_cluster,center,seed_links):
    '''
    input: 
        network_link_list: a list of link id that needs to be partitioned 
        num_cluster: the network would be partitioned into num_cluster sets
        center: optional input, the location of the depot
        seed_links: optional input, the seed link for each cluster
    output
        clusters: a list of link contained in each cluster
    '''
#     global d
    if not center: 
        centertemp=random.choice(network_link_list)
        center=link_detail[centertemp,0]
    if not seed_links: # in this case, we do not have a seedlink,but have a center depot
        for i in range(num_cluster):
            maxdistance=0
            linkcandidate=0
            #Go over all the candidate links to find the one which locates furtherest to the depot and other seedlinks
            for link in network_link_list:
                #calculate distance form link candidate to the depot
#                 Distancecalculated based on the middle points of the link
#                 distance_to_depot=math.sqrt(((nodexy[link_detail[link,0]][0]+nodexy[link_detail[link,1]][0])/2-nodexy[center][0])**2+
#                                             ((nodexy[link_detail[link,0]][1]+nodexy[link_detail[link,1]][1])/2-nodexy[center][1])**2) 
                #Distance calculated based on the up stream node
                distance_to_depot=math.sqrt((nodexy[link_detail[link,0]][0]-nodexy[center][0])**2+
                                             (nodexy[link_detail[link,0]][1]-nodexy[center][1])**2) 
                total_distance=distance_to_depot
                #calculate distance form link candidate to other seed links
                for seed_link in seed_links: 
                    total_distance=total_distance*distance_between_link(seed_link,link,link_xy)
                if total_distance>maxdistance and link not in seed_links:
                    maxdistance=total_distance
                    linkcandidate=link
            seed_links.append(linkcandidate)
    print('The following links are selected as seed link: \n',seed_links)
    
    #After the seed_links are selected, solve the integer problem to identify the link clusters
    cluster_link_index=[i for i in range(len(network_link_list))]
    cluster_zone_index=[i for i in range(num_cluster)]
    link_length=[link_detail[i,4] for i in network_link_list]
    
#     L=float(sum(link_length)/num_cluster*0.8)
#     U=float(sum(link_length)/num_cluster*1.2)

#     prob=pulp.LpProblem('Link Cluster Problem',pulp.LpMinimize)
#     link_cluster=pulp.LpVariable.dicts("Cluster",(cluster_link_index,cluster_zone_index),0,1,pulp.LpInteger)
    print('Prepare distance matrix between links and seed links')
    d=np.zeros(shape=(len(network_link_list),num_cluster))
    for l in range(len(network_link_list)):
        for z in range(num_cluster):
            #print(l,z,network_link_list[l],seed_links[z])
            d[l,z]=distance_between_link(network_link_list[l],seed_links[z],link_xy)
    
#     prob += pulp.lpSum([d[l,z]*link_cluster[l][z]] for l in cluster_link_index for z in cluster_zone_index), 
#"Total distance from link to assigned seed link"
#     for l in cluster_link_index:
#         prob += pulp.lpSum([link_cluster[l][z] for z in cluster_zone_index]) == 1
#     for z in cluster_zone_index:
#         prob += pulp.lpSum([float(link_length[l])*link_cluster[l][z]] for l in cluster_link_index )>=L
#         prob += pulp.lpSum([float(link_length[l])*link_cluster[l][z]] for l in cluster_link_index)<=U
        
#     prob.writeLP("Link Cluster Problem.lp")
#     # The problem is solved using PuLP's choice of Solver
#     prob.solve()
#     print("Solution Status:", pulp.LpStatus[prob.status])
    network_link_cluster_list=[[] for i in range(num_cluster)]
    for l in cluster_link_index:
        s_distance=999999
        c_seed=99
        for z in cluster_zone_index:
            if d[l,z]<s_distance:
                s_distance=d[l,z]
                c_seed=z
        network_link_cluster_list[c_seed].append(network_link_list[l])
        
#             if pulp.value(link_cluster[l][z])==1:
#                 network_link_cluster_list[z].append(network_link_list[l])
            #Added for the nearest heuristic
            
    
    return network_link_cluster_list


In [ ]:
# def multi_commodity_model(cluster):
#     '''
#         input:
#             clusters: link ID of links will be covered by the depot
#             link: link length of all links within the clusters
#             s_speed: service speed
#             d_speed: deadhead speed
#         intermediat output: 
#             x: 0 or 1 integer indicate whether an arc is serviced or not
#             y: an integer indicates whether the edge is traversed by associated vehicle
#         output: 
#             route plan
#     '''
#     #Reconstruct the network
#     #Duplicate multiple lane 
#     new_cluster=[]
#     ijindicator=[]
#     for link in cluster: 
#         for lane in range(0,int(link_detail[link,5])):
#             new_cluster.append(link)
#             if link_detail[link,0]>link_detail[link,2]:
#                 ijindicator.append(1)
#             else:
#                 ijindicator.append(0)
#     #Add the depot and fictious class (needs to be done)
    
#     #Create the index for the vehicle class
#     cluster_link_index=[i for i in range(len(new_cluster))]
#     cluster_class_index=[i for i in range(3)]
    
#     #decision variables
#     service_links=pulp.LpVariable.dicts("service_link",(cluster_link_index,cluster_class_index),0,1,pulp.LpInteger)
#     traversed_links=pulp.LpVariable.dicts("traversed_link",(cluster_link_index,cluster_class_index),0,10,pulp.LpInteger)
    
#     prob=pulp.LpProblem('Snowplowrouting Problem',pulp.LpMinimize)
    
#     #input variables
#     service_time=[link_detail[i,4]/1024/s_speed for i in new_cluster]
#     deadhead_time=[link_detail[i,4]/1024/d_speed for i in new_cluster]
#     #objective function
#     prob += pulp.lpSum(service_links[l][c]*float(service_time[l])+traversed_links[l][c]*float(deadhead_time[l]) for l in cluster_link_index for c in cluster_class_index)
    
#     #constraints 2.2
#     for p in cluster_class_index:
#         if p==0:
#             prob += pulp.lpSum(service_links[l][0]*float(service_time[l])+traversed_links[l][0]*float(deadhead_time[l]) for l in cluster_link_index)>=0.0
#         else:
#             prob += pulp.lpSum(service_links[l][p]*float(service_time[l])+traversed_links[l][p]*float(deadhead_time[l]) for l in cluster_link_index)-pulp.lpSum(service_links[l][p-1]*float(service_time[l])+traversed_links[l][p-1]*float(deadhead_time[l]) for l in cluster_link_index)>=0.01
#     #Completion time 2.3
#     for l in cluster_link_index:
#         print(pulp.lpSum([service_links[l][z] for z in cluster_class_index]))
#         prob += pulp.lpSum([service_links[l][z] for z in cluster_class_index]) == 1  #2.5
#     #2.7
#     prob += pulp.lpSum(service_links[l][z]-2*service_links[l][z]*ijindicator[l] for l in cluster_link_index for z in cluster_class_index)==0
#     prob += pulp.lpSum(service_links)
#     print(service_links[l][z]*ijindicator[l] for l in cluster_link_index) #for z in cluster_class_index)
#     #2.8
    
    
    
#     prob.writeLP("Multi Commodity Routing.lp")
#     # The problem is solved using PuLP's choice of Solver
#     prob.solve()
#     print("Solution Status:", pulp.LpStatus[prob.status])
    
# #     network_link_cluster_list=[[] for i in range(num_cluster)]
# #     for l in cluster_link_index:
# #         for z in cluster_zone_index:
# #             if pulp.value(link_cluster[l][z])==1:
# #                 network_link_cluster_list[z].append(network_link_list[l])
    
#     return
    
    
    

In [ ]:
# Initial settings and read inputs
print('Read the network.dat')
read_network()
# Read the traffic volume outflow.dat
# print('Read the OutFlow.dat')
read_flow()
read_speed()
print('Read the xy.dat')
read_xy()
# Parameter Setting

# The input
global s_speed,d_speed,num_simulation_interval,simulation_length,planning_horizon
s_speed= 0.5 #mile per minute
d_speed= 0.5 #mile per minute
Num_Zones=4
Num_Snowplows=10
num_link_classes=3
depots=[14522]
num_simulation_interval=1440/5
simulation_length=5.0
planning_horizon=1440.0
VOT=5 # $/minute
OpCost=0.25 # $/minute
link_id=[i for i in range(num_links)]
print('Read the snow information')
read_snow()
read_snowaccum()
c_link_xy(link_id)
#Intial Value for the service finish time of each lane 
service_finished_time=[]
i=0
for link in link_detail:
    service_finished_time.append([])
    service_finished_time[i].extend([planning_horizon]*link[0,5])
    i=i+1


In [ ]:
# Divided the link into different catogories
# print('Calibrate Service Class')
# link_class_partrition()

#Determine the subzone covered by each depot
print('Determing subzone covered by depot each depot')
#sub_service_zone=net_work_partrition(subzone,len(depots),[],depots)
sub_service_zone=net_work_partrition(link_id,Num_Zones,depots[0],[])

#Construct sub_cluster within each subzone
print('Determing subcluster covered by each snowplow')
#sub_service_clusters=net_work_partrition(subzone,Num_Snowplows,depots[0],[])

#sub_vehicle_clusters=network_partrition_nearest_heuristic(sub_service_zone[2],Num_Snowplows,[],[])
# Locating M geographically dispersed arcs of A to serve as seed arcs for the M vehicles
# Partrition the network into /Num_Snowplowss/ subarea. 
# for cluster in sub_service_clusters:
#     #Solve the linear integer for each cluster
#     check_temp=1
    

print('Assign links to specified vehicle')


In [ ]:
#sub_service_cluster=net_work_partrition(sub_service_zone[2],Num_Snowplows,14103,[])

In [ ]:
#sub_service_cluster=network_partrition_nearest_heuristic(sub_service_zone[2],Num_Snowplows,14103,[])

In [ ]:
def cluster_highlight(linklist):
    #This function will highlight the linklist in a given network
    plt.figure() 
    Gnormal=nx.Graph()
    Gred=nx.Graph()
    read_network()
    read_xy()

    for nodeinf in node_detail:
        node=nodeinf[0,0]
        Gnormal.add_node(node,pos=(nodexy[node][0],nodexy[node][1]))
        Gred.add_node(node,pos=(nodexy[node][0],nodexy[node][1]))
#         Gnormal.add_node(node,pos=(nodexy[node][1],nodexy[node][0]))
#         Gred.add_node(node,pos=(nodexy[node][1],nodexy[node][0]))
    counter=0
    red_link=[]
    normal_link=[]
    for linkinf in link_detail: 
        if counter in linklist:
            red_link.append((linkinf[0,0],linkinf[0,1]))
            Gred.add_edge(linkinf[0,0],linkinf[0,1])        
        else:
            normal_link.append((linkinf[0,0],linkinf[0,1]))
            Gnormal.add_edge(linkinf[0,0],linkinf[0,1])
        counter=counter+1
            
    pos=nx.get_node_attributes(Gnormal,'pos')
    nx.draw(Gnormal,pos,node_size=1)
    nx.draw(Gred,pos,edge_color='r',width=4,node_size=4)
    #nx.draw_networkx_nodes(G,pos,node_size=50)
    plt.draw()
   
    plt.show()
    return

In [ ]:
def snow_penalty(service_finished_time,snow_detail,sa_factors):
    '''
    input: 
        service_finished_time: the expected service finished time of each link
        snow_precipitation: The snow precipitation rate 
        sa_factors: The speed/capacity reduction rate caused by different level of snow accumulation
    Output:   
        Tot_Delay: delay due to the snow accumulation
    Intermediate variable: 
        snow_depth[]: snowdepth of each lane at current time interval
    '''
    average_snow_depth=[]
    j=0
    Tot_Delay=0
    for link in link_detail: #Calculate the penalty on each link
        #Calculate the reduced speed based on the snow depth
        st =service_finished_time[j]
        snow_depth_temp=0
        snow_depth=[0]*math.floor(link[0,5])
        for t in range (math.floor(num_simulation_interval)):
            penaltytemp=0
            ####Calculate the snow depth
            for snow in snow_detail:
                if snow[2] > (t+0.5)*simulation_length and snow[1] < (t+0.5)*simulation_length:
                    current_snow=snow[0]
            for i in range(math.floor(link[0,5])):
                if service_finished_time[j][i] <(t+1)*simulation_length and service_finished_time[j][i] >t*simulation_length:
                    snow_depth[i]=0
                else:
                    snow_depth[i]=snow_depth[i]+current_snow*snow_interval/60
            average_snow_depth=sum(snow_depth)/link[0,5]
            ####Finish calculating the snow depth
            
            ### Calculate the affected speed
            origin_speed=link_speed[math.floor(t*simulation_length),j]
            k=0
            while average_snow_depth>sa_factors[k][0]:
                k=k+1
            speed_reduction=sa_factors[k][1]
            capacity_reduction=sa_factors[k][2]
            reduced_speed=origin_speed*speed_reduction #account for the speed reduction
            #reduced_speed= #account for the capacity reduction use the BPR function
            penaltytemp=link_volume[math.floor(t*simulation_length),j]*(origin_speed-reduced_speed)/origin_speed*simulation_length
            Tot_Delay=Tot_Delay+penaltytemp
        j=j+1
    return Tot_Delay
#def sp_cost(start_node,target_node)
def link_snow_penalty (link,service_finished_time):
    '''
    This subroutine update the total snow_penalty when only service time of one link is updated

    input: 
        service_finished_time: the expected service finished time of each link
        snow_precipitation: The snow precipitation rate 
        sa_factors: The speed/capacity reduction rate caused by different level of snow accumulation
    Output:   
        Tot_Delay: delay due to the snow accumulation
    Intermediate variable: 
        snow_depth[]: snowdepth of each lane at current time interval
    '''
    #Calculate the reduced speed based on the snow depth
    link_delay=0
    snow_depth_temp=0
    snow_depth=[0]*math.floor(link_detail[link,5])
    for t in range (math.floor(num_simulation_interval)):
        penaltytemp=0
        ####Calculate the snow depth
        for snow in snow_detail:
            if snow[2] > (t+0.5)*simulation_length and snow[1] < (t+0.5)*simulation_length:
                current_snow=snow[0]
        for i in range(math.floor(link_detail[link,5])):
            if service_finished_time[link][i] <=(t+1)*simulation_length and service_finished_time[link][i] >=t*simulation_length:
                snow_depth[i]=0
            else:
                snow_depth[i]=snow_depth[i]+current_snow*snow_interval/60
        average_snow_depth=sum(snow_depth)/link_detail[link,5]
        
        ####Finish calculating the snow depth

        ### Calculate the affected speed
        origin_speed=link_speed[math.floor(t*simulation_length),link]
        k=0
        while average_snow_depth>sa_factors[k][0]:
            k=k+1
        speed_reduction=sa_factors[k][1]
        capacity_reduction=sa_factors[k][2]
        reduced_speed=origin_speed*speed_reduction #account for the speed reduction
        #reduced_speed= #account for the capacity reduction use the BPR function
        penaltytemp=link_volume[math.floor(t*simulation_length),link]*(origin_speed-reduced_speed)/origin_speed*simulation_length
        link_delay=link_delay+penaltytemp
    return link_delay
    

def calculate_service_finished_time ():
    '''
        Input: 
            Snowroutes: A list of nodes indicating the order of snow plow service
        Output: 
            service_finished_time: The service finish time of each lane
    '''
    return
# def path_scan(DepotLocation,LinkList):
#     '''
#     Input: 
#     DepotLocation: The node number of the depot
#     LinkList: The list of link to be plowed by the snowplow
#     Output: 
#     RoutePlan： A list of nodes indicating the working order of snow plow
#     '''
#     #Select seed node 
#     seed_node=random.choice(LinkList)
    
#     #Calculate the objective function
#     objvalue=snow_penalty()
#     #For all different priority classes construct the inital routes
#         #Set P empty set
#         #for all links
#             #Find the arc ij closet to depot/end node of class k-1 while optimize objective function
#             #If found set end=j
#         #Set P=(P,ij)
#         #for all (end,i) 
#             #find the end,i that maximizes the number of non-serviced required arc of classs k adjacent to vei
#             #set endtemp=i
#         #if endtemp ==0 then 
#             #P=P+SP(end,0)
#             #else
#             #P=P+(end,endtemp)
#             #end= endtemp
#     #Then for all non-serviced arcs ij
#         #add iji to the routes
#     #Then for all non serviced arc ij
#         #find the insertion place minimize bjective function 
#     return RoutePlan
def C_link_to_link_distance(ServiceZone):
    '''
        input: ServiceZone: sorted list of link ID, indicating the links that will be served
    '''
    link_to_link=[[0 for x in range(len(ServiceZone))] for y in range(len(ServiceZone))] 
    i=0
    link_to_link={}
    for linko in ServiceZone:
        link_to_link[linko]={}
        for linkd in ServiceZone[i+1:]:
            distance=distance_between_link(linko,linkd,link_xy)
            link_to_link[linko].update({linkd:distance})
        i=i+1
    return link_to_link



In [ ]:
def path_scan_heuristic(DepotLocation,ServiceZone,service_start_time):
    '''
    Input: 
        DepotLocation: The node number of the depot
        LinkList: The list of link to be plowed by the snowplow
        service_start_time: The time the first snowplow leave the depot
    Output: 
        RoutePlan: A list of links indicating the working order of snow plow
    '''
    global non_serviced_arcs,RoutePlan
    ServiceZone=sorted(ServiceZone)
    minobj=9*10**9
    
    non_serviced_arcs=[]
    RoutePlan=[]
    #Create a link to link distance matrix
    link_to_link=C_link_to_link_distance(ServiceZone)
        
    #Duplicate arcs with i lanes into i links
    for link in ServiceZone: 
        for i in range(math.floor(link_detail[link,5])):
            non_serviced_arcs.extend([link])
    num_served_arcs=0
    link_delay={}
    for link in ServiceZone:
        link_delay.update({link:link_snow_penalty(link,service_finished_time)})

    while non_serviced_arcs:
        output_file=open('../test1.csv','a')
        print('There are',num_served_arcs,'ars have been served')
        print('Find the',num_served_arcs,'arc',file=output_file)
        
        linktemp=9000000
        for link in non_serviced_arcs:
            if link !=linktemp:
                linktemp=link
                target_lane=math.floor(float(np.argmax(service_finished_time[link])))
                #Find the link that minimize the objective function
                
                    
                if not RoutePlan: #if the route plan is empty, then identify the first link
                    sv_time_temp=service_finished_time[link][target_lane]
                    #Calculate the travel time from current node to the target link
                    #print(0,link,minobj,link_delay[link],sum(link_delay.values()) *VOT,service_finished_time[link])
                    if link_detail[link,0] != DepotLocation:
                        distance_temp=math.sqrt((link_xy[link][0]-nodexy[DepotLocation][0])**2+(link_xy[link][1]-nodexy[DepotLocation][1])**2)*1.3
                        service_finished_time[link][target_lane]=service_start_time+distance_temp/d_speed+link_detail[link,4]/s_speed/5280
                    else:
                        distance_temp=0
                        service_finished_time[link][target_lane]=service_start_time+link_detail[link,4]/s_speed/5280
                    #print(0.1,link,minobj,link_delay[link],sum(link_delay.values()) *VOT,service_finished_time[link])
                    link_delay_temp=link_delay[link]    
                    link_delay[link]=link_snow_penalty(link,service_finished_time)
                    objtemp=sum(link_delay.values()) *VOT+distance_temp/d_speed*OpCost 
                    if objtemp<minobj:
                        minobj=objtemp
                        LinkCandidate=link
                        can_serviced_time=service_finished_time[link][target_lane]
                        can_delay_time=link_snow_penalty(link,service_finished_time)
                        print(1,link,minobj,objtemp,can_delay_time,link_delay[link],sum(link_delay.values()) *VOT,service_finished_time[link],file=output_file)
                    service_finished_time[link][target_lane]=sv_time_temp
                    link_delay[link]=link_delay_temp
                    if objtemp==minobj:
                        print(2,link,minobj,objtemp,can_delay_time,link_delay[link],sum(link_delay.values()) *VOT,service_finished_time[link],file=output_file)
                elif link_detail[link,1]!= link_detail[RoutePlan[-1],0] and link != RoutePlan[-1]:
                    sv_time_temp=service_finished_time[link][target_lane]
                    #Calculate the travel time from current node to the target link
                    if link_detail[link,0] != link_detail[RoutePlan[-1],1]:
                        distance_temp=link_to_link[min(RoutePlan[-1],link)][max(RoutePlan[-1],link)]*1.3
                        service_finished_time[link][target_lane]=service_start_time+distance_temp/d_speed+link_detail[link,4]/s_speed/5280
                    else:
                        distance_temp=0
                        service_finished_time[link][target_lane]=service_start_time+link_detail[link,4]/s_speed/5280
                    #print(3,link,minobj,sum(link_delay.values()) *VOT,distance_temp/d_speed*OpCost,link_delay[link])
                    link_delay_temp=link_delay[link]    
                    link_delay[link]=link_snow_penalty(link,service_finished_time)
                    #print(3.1,link,minobj,sum(link_delay.values()) *VOT,distance_temp/d_speed*OpCost,link_delay[link])
                    objtemp=sum(link_delay.values()) *VOT+distance_temp/d_speed*OpCost

    #                 pdb.set_trace()
                    if objtemp<minobj:
                        minobj=objtemp
                        LinkCandidate=link
                        can_serviced_time=service_finished_time[link][target_lane]
                        print(3,link,minobj,objtemp,can_delay_time,link_delay[link],sum(link_delay.values()) *VOT,service_finished_time[link],file=output_file)
                    service_finished_time[link][target_lane]=sv_time_temp
                    link_delay[link]=link_delay_temp
                    if objtemp==minobj:
                        print(4,link,minobj,objtemp,can_delay_time,link_delay[link],sum(link_delay.values()) *VOT,service_finished_time[link],file=output_file)
        #Updated the route plan, service finished time. Remove selected link from non serviced arcs list
        
        print(5,LinkCandidate,minobj,can_delay_time,link_delay[LinkCandidate],sum(link_delay.values()) *VOT,service_finished_time[LinkCandidate],file=output_file)
        RoutePlan.extend([LinkCandidate])
        service_finished_time[LinkCandidate][target_lane]=can_serviced_time
#         if LinkCandidate==139:
#             pdb.set_trace()
#         if 139 not in non_serviced_arcs: 
#             print('stope1')
#             pdb.set_trace()
        if LinkCandidate not in non_serviced_arcs: 
            print('The current link candidate is not a non serviced required arc')
            pdb.set_trace()
        non_serviced_arcs.remove(LinkCandidate)
#         if 139 not in non_serviced_arcs: 
#             print('stope1')
#             pdb.set_trace()
        service_start_time=can_serviced_time
        link_delay[LinkCandidate]=link_snow_penalty(LinkCandidate,service_finished_time)
        num_served_arcs=num_served_arcs+1
        print('Link',LinkCandidate,'is added to the route.','And the service time is',can_serviced_time)
        print(6,str(datetime.now()),LinkCandidate,minobj,can_delay_time,link_delay[LinkCandidate],sum(link_delay.values()) *VOT,service_finished_time[LinkCandidate],file=output_file)
        #pdb.set_trace()
        output_file.close()
    
    return

In [ ]:
service_finished_time=[]
i=0
for link in link_detail:
    service_finished_time.append([])
    service_finished_time[i].extend([planning_horizon]*math.floor(link[0,5]))
    i=i+1

from datetime import datetime
path_scan_heuristic(13339,sub_service_zone[0],480)

In [ ]:
a=[2,3,4,6]
if 5 not in a:
    print('yes')
           

In [ ]:
non_serviced_arcs.remove(139)


In [ ]:
non_serviced_arcs